# The Event and Schedule classes

You should copy this code into your own Python notebook and use it to complete your pre-class work.

In [1]:
import heapq

class Event:
    '''
    Store the properties of one event in the Schedule class defined below. Each
    event has a time at which it needs to run, a function to call when running
    the event, along with the arguments and keyword arguments to pass to that
    function.
    '''
    def __init__(self, timestamp, function, *args, **kwargs):
        self.timestamp = timestamp
        self.function = function
        self.args = args
        self.kwargs = kwargs

    def __lt__(self, other):
        '''
        This overloads the less-than operator in Python. We need it so the
        priority queue knows how to compare two events. We want events with
        earlier (smaller) times to go first.
        '''
        return self.timestamp < other.timestamp

    def run(self, schedule):
        '''
        Run an event by calling the function with its arguments and keyword
        arguments. The first argument to any event function is always the
        schedule in which events are being tracked. The schedule object can be
        used to add new events to the priority queue.
        '''
        self.function(schedule, *self.args, **self.kwargs)


class Schedule:
    '''
    Implement an event schedule using a priority queue. You can add events and
    run the next event.
    
    The `now` attribute contains the time at which the last event was run.
    '''
    
    def __init__(self):
        self.now = 0  # Keep track of the current simulation time
        self.priority_queue = []  # The priority queue of events to run
    
    def add_event_at(self, timestamp, function, *args, **kwargs):
        # Add an event to the schedule at a particular point in time.
        heapq.heappush(
            self.priority_queue,
            Event(timestamp, function, *args, **kwargs))
    
    def add_event_after(self, interval, function, *args, **kwargs):
        # Add an event to the schedule after a specified time interval.
        self.add_event_at(self.now + interval, function, *args, **kwargs)
    
    def next_event_time(self):
        # Return the time of the next event. The `now` attribute of this class
        # contain the time of the last event that was run.
        return self.priority_queue[0].timestamp

    def run_next_event(self):
        # Get the next event from the priority queue and run it.
        event = heapq.heappop(self.priority_queue)
        self.now = event.timestamp
        event.run(self)
        
    def __repr__(self):
        return (
            f'Schedule() at time {self.now} ' +
            f'with {len(self.priority_queue)} events in the queue')
    
    def print_events(self):
        # Print out diagnostic information about the events in the schedule.
        print(repr(self))
        for event in sorted(self.priority_queue):
            print(f'   {event.timestamp}: {event.function.__name__}')

## A simple test of the schedule

You do not need this code for your pre-class work. It demonstrates that the schedule can add and run events.

In [2]:
def print_and_add(schedule, string):
    print(f'At time {schedule.now}:', string)
    schedule.add_event_after(0.25, print_and_add, 'Another event')
            
schedule = Schedule()
schedule.print_events()

schedule.add_event_at(0.5, print_and_add, 'First event')
schedule.add_event_at(1.2, print_and_add, 'Second event')
schedule.print_events()

schedule.run_next_event()
schedule.print_events()

schedule.run_next_event()
schedule.print_events()

schedule.run_next_event()
schedule.print_events()

Schedule() at time 0 with 0 events in the queue
Schedule() at time 0 with 2 events in the queue
   0.5: print_and_add
   1.2: print_and_add
At time 0.5: First event
Schedule() at time 0.5 with 2 events in the queue
   0.75: print_and_add
   1.2: print_and_add
At time 0.75: Another event
Schedule() at time 0.75 with 2 events in the queue
   1.0: print_and_add
   1.2: print_and_add
At time 1.0: Another event
Schedule() at time 1.0 with 2 events in the queue
   1.2: print_and_add
   1.25: print_and_add


# The cuckoo clock simulation

You do not need this code for your pre-class work but you can use it as a starting point for implementing your queue simulation.

In [3]:
class Cuckoo:
    def __init__(self, interval):
        self.interval = interval
        self.hour = 0
    
    def make_some_noise(self, schedule):
        self.hour += 1
        print('Cuckoo! ' * self.hour)
        if self.hour == 12:
            self.hour = 0

class Pendulum:
    def __init__(self, cuckoo):
        self.cuckoo = cuckoo
    
    def tick(self, schedule, counter):
        counter += 1
        print(f'tick {counter} at time = {schedule.now}')
        schedule.add_event_after(0.5, self.tock, counter)
        
    def tock(self, schedule, counter):
        print(f'tock {counter} at time = {schedule.now}')
        schedule.add_event_after(0.5, self.tick, counter)
        if counter % self.cuckoo.interval == 0:
            schedule.add_event_after(0, self.cuckoo.make_some_noise)

class Clock:
    def __init__(self):
        self.cuckoo = Cuckoo(interval=5)
        self.pendulum = Pendulum(self.cuckoo)
    
    def run(self, schedule):
        schedule.add_event_at(0.5, self.pendulum.tick, counter=0)

def run_simulation(run_until):
    schedule = Schedule()
    clock = Clock()
    clock.run(schedule)
    while schedule.next_event_time() <= run_until:
        schedule.run_next_event()

In [4]:
run_simulation(run_until=10)

tick 1 at time = 0.5
tock 1 at time = 1.0
tick 2 at time = 1.5
tock 2 at time = 2.0
tick 3 at time = 2.5
tock 3 at time = 3.0
tick 4 at time = 3.5
tock 4 at time = 4.0
tick 5 at time = 4.5
tock 5 at time = 5.0
Cuckoo! 
tick 6 at time = 5.5
tock 6 at time = 6.0
tick 7 at time = 6.5
tock 7 at time = 7.0
tick 8 at time = 7.5
tock 8 at time = 8.0
tick 9 at time = 8.5
tock 9 at time = 9.0
tick 10 at time = 9.5
tock 10 at time = 10.0
Cuckoo! Cuckoo! 


# M/D/1 queue

In [5]:
import scipy.stats as sts

class Queue:
    def __init__(self, service_rate):
        # Store the deterministic service time for an M/D/1 queue
        self.service_time = 1 / service_rate
        # We start with an empty queue and the server not busy
        self.people_in_queue = 0
        self.people_being_served = 0

    def add_customer(self, schedule):
        # Add the customer to the queue
        self.people_in_queue += 1
        print(
            f'{schedule.now:5.2f}: Add customer to queue.  '
            f'Queue length: {self.people_in_queue}')
        if self.people_being_served < 1:
            # This customer can be served immediately
            schedule.add_event_after(0, self.start_serving_customer)
            
    def start_serving_customer(self, schedule):
        # Move the customer from the queue to a server
        self.people_in_queue -= 1
        self.people_being_served += 1
        print(
            f'{schedule.now:5.2f}: Start serving customer. '
            f'Queue length: {self.people_in_queue}')
        # Schedule when the server will be done with the customer
        schedule.add_event_after(
            self.service_time,
            self.finish_serving_customer)
            
    def finish_serving_customer(self, schedule):
        # Remove the customer from the server
        self.people_being_served -= 1
        print(
            f'{schedule.now:5.2f}: Stop serving customer.  '
            f'Queue length: {self.people_in_queue}')
        if self.people_in_queue > 0:
            # There are more people in the queue so serve the next customer
            schedule.add_event_after(0, self.start_serving_customer)


class BusSystem:
    
    def __init__(self, arrival_rate, service_rate):
        self.queue = Queue(service_rate)
        self.arrival_distribution = sts.expon(scale=1/arrival_rate)

    def add_customer(self, schedule):
        # Add this customer to the queue
        self.queue.add_customer(schedule)
        # Schedule when to add another customer
        schedule.add_event_after(
            self.arrival_distribution.rvs(),
            self.add_customer)

    def run(self, schedule):
        # Schedule when the first customer arrives
        schedule.add_event_after(
            self.arrival_distribution.rvs(),
            self.add_customer)
        

def run_simulation(arrival_rate, service_rate, run_until):
    schedule = Schedule()
    bus_system = BusSystem(arrival_rate, service_rate)
    bus_system.run(schedule)
    while schedule.next_event_time() < run_until:
        schedule.run_next_event()
    return bus_system

In [8]:
arrival_rate = 1
service_rate = 1.5
duration = 50
bus_system = run_simulation(arrival_rate, service_rate, duration)
print(f'There are {bus_system.queue.people_in_queue} people in the queue')

 0.28: Add customer to queue.  Queue length: 1
 0.28: Start serving customer. Queue length: 0
 0.45: Add customer to queue.  Queue length: 1
 0.80: Add customer to queue.  Queue length: 2
 0.95: Stop serving customer.  Queue length: 2
 0.95: Start serving customer. Queue length: 1
 1.51: Add customer to queue.  Queue length: 2
 1.62: Stop serving customer.  Queue length: 2
 1.62: Start serving customer. Queue length: 1
 1.65: Add customer to queue.  Queue length: 2
 1.73: Add customer to queue.  Queue length: 3
 1.95: Add customer to queue.  Queue length: 4
 2.28: Stop serving customer.  Queue length: 4
 2.28: Start serving customer. Queue length: 3
 2.69: Add customer to queue.  Queue length: 4
 2.95: Stop serving customer.  Queue length: 4
 2.95: Start serving customer. Queue length: 3
 3.50: Add customer to queue.  Queue length: 4
 3.59: Add customer to queue.  Queue length: 5
 3.62: Stop serving customer.  Queue length: 5
 3.62: Start serving customer. Queue length: 4
 4.16: Add cu